In [81]:
# 가로로 넓게 쓰기
from IPython.core.display import display, HTML  
display(HTML(""))

print("Setup Complete")

Setup Complete


In [82]:
import pandas as pd
import numpy as np
import os

import time 
import datetime as dt
from datetime import datetime

from sklearn.preprocessing import LabelEncoder

In [83]:
# def make_df(df, label):
    
#     for col in label:
#         print(col)
#         le = LabelEncoder()
#         k = df.pop(col)
#         le.fit(k)
#         k = pd.DataFrame(le.transform(k))
#         df = pd.concat([df, k], axis=1)
        
#     return df

In [84]:
def generate_features(df):
    df['STD_DT'] = df['STD_DT'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df.loc[:, 'weekofyear'] = df['STD_DT'].dt.weekofyear
    df.loc[:, 'dayofweek'] = df['STD_DT'].dt.dayofweek
    df.loc[:, 'weekend'] = (df['STD_DT'].dt.weekday >=5).astype(int)
#     df.loc[:, 'hour'] = df['STD_DT'].dt.hour
#     df.loc[:, 'min'] = df['STD_DT'].dt.mimute
    
    return df

In [85]:
path = "/home/kym/ML/input/kamp/대회 과제"
os.chdir(path)

df = pd.read_csv('melting_tank_mod.csv', encoding='cp949')

df = generate_features(df)
feature = [
    f for f in df.columns if f in ("STD_DT", "MELT_TEMP", "MOTORSPEED", "MELT_WEIGHT", "weekofyear", "dayofweek", "weekend", "TAG")
]

df = df[feature]
# df = make_df(df, ['TAG'])
# df.drop([0], axis=1, inplace=True)
df.to_csv('/home/kym/ML/input/kamp/대회 과제/melting_tank_final.csv', index=False)
df.head()

/tmp/ipykernel_16398/1836040535.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df.loc[:, 'weekofyear'] = df['STD_DT'].dt.weekofyear


,STD_DT,MELT_TEMP,MOTORSPEED,MELT_WEIGHT,TAG,weekofyear,dayofweek,weekend
0,2020-03-04,48.9,11.6,631,1,10,2,0
1,2020-03-04,43.3,7.8,609,1,10,2,0
2,2020-03-04,46.4,15.4,608,1,10,2,0
3,2020-03-04,37.9,21.2,606,1,10,2,0
4,2020-03-04,79.8,173.6,604,1,10,2,0


In [86]:
tag = pd.DataFrame(df['TAG'])
tag

,TAG
0,1
1,1
2,1
3,1
4,1
...,...
835195,1
835196,1
835197,1
835198,1


In [91]:
# 1 정상, 0 불량 (가이드 북 기준, 본래는 유저가 관심 있는 것을 보통 1 로 사용)
print("정상 개수 : {} ({:.2f} %)".format(tag.TAG.value_counts()[1], tag.TAG.value_counts(normalize=True)[1]))
print("비 정상 개수: {} ({:.2f} %)".format(tag.TAG.value_counts()[0], tag.TAG.value_counts(normalize=True)[0])) 

정상 개수 : 658133 (0.79 %)
비 정상 개수: 177067 (0.21 %)
